In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score

# Load the interaction data
data_path = 'metadata/'
interaction_df = pd.read_csv(data_path + 'interactions.csv').drop('Unnamed: 0', axis=1)
interaction_df = interaction_df.rename(columns={'FACILITYCODE': 'FacilityCode'})

# Preprocess the interaction data
# Preprocess the interaction data
interaction_df['SurveyAnswerScore'] = np.round(interaction_df['SurveyAnswerScore'], 2)
interaction_df['RecommendStar'] = np.round(interaction_df['RecommendStar'], 2)
interaction_df = interaction_df.drop('InsertedOn', axis=1)

# Aggregate the interaction data
interaction_df_mod = interaction_df.groupby(['PERSON_ID', 'ProviderCode']).agg({
    'SurveyAnswerScore': 'mean',
    'RecommendStar': 'mean',
    'num_visits': 'first',
    'age': 'mean',
    'ratings': 'mean',
    'star_rating': 'first',
    'survey_score': 'first',
    'zip_code': 'first',
    'GENDER_Female': 'first',
    'GENDER_Male': 'first',
    'FacilityCode': lambda x: list(x),
    'SpecialtyCode': lambda x: list(x)
}).reset_index()

interaction_df_mod['visit_ratings'] = 0.65 * interaction_df_mod['ratings'] + 0.35 * interaction_df_mod['num_visits']

# Generate unique integer mappings for FacilityCode and SpecialtyCode
all_facility_codes = set(code for sublist in interaction_df_mod['FacilityCode'] for code in sublist)
facility_code_map = {code: idx for idx, code in enumerate(all_facility_codes)}

all_specialty_codes = set(code for sublist in interaction_df_mod['SpecialtyCode'] for code in sublist)
specialty_code_map = {code: idx for idx, code in enumerate(all_specialty_codes)}

interaction_df_mod['FacilityCode'] = interaction_df_mod['FacilityCode'].apply(lambda codes: [facility_code_map[code] for code in codes])
interaction_df_mod['SpecialtyCode'] = interaction_df_mod['SpecialtyCode'].apply(lambda codes: [specialty_code_map[code] for code in codes])

# Prepare provider features
provider_features = interaction_df_mod.groupby('ProviderCode').agg({
    'num_visits': 'sum',
    'survey_score': 'mean',
    'star_rating': 'mean',
    'FacilityCode': lambda x: list(set([i for sublist in x for i in sublist])),
    'SpecialtyCode': lambda x: list(set([i for sublist in x for i in sublist]))
}).reset_index()

# Prepare person features
person_features = interaction_df_mod.groupby('PERSON_ID').agg({
    'age': 'first',
    'GENDER_Female': 'first',
    'GENDER_Male': 'first',
    'FacilityCode': lambda x: list(set([i for sublist in x for i in sublist])),
    'SpecialtyCode': lambda x: list(set([i for sublist in x for i in sublist]))
}).reset_index()

# Prepare edge features
edges = interaction_df_mod[['PERSON_ID', 'ProviderCode', 'visit_ratings']]

# Map IDs to integers
person_id_map = {id: idx for idx, id in enumerate(person_features['PERSON_ID'].unique())}
provider_id_map = {id: idx + len(person_id_map) for idx, id in enumerate(provider_features['ProviderCode'].unique())}
id_provider_map = {value: key for key, value in provider_id_map.items()}

# Apply mapping to the edges
edges['PERSON_ID'] = edges['PERSON_ID'].map(person_id_map)
edges['ProviderCode'] = edges['ProviderCode'].map(provider_id_map)

# Initialize the graph
G = nx.Graph()

# Add person nodes with features
for _, row in person_features.iterrows():
    G.add_node(person_id_map[row['PERSON_ID']], bipartite=0, age=row['age'], gender_female=row['GENDER_Female'], gender_male=row['GENDER_Male'], facility_codes=row['FacilityCode'], specialty_codes=row['SpecialtyCode'])

# Add provider nodes with features
for _, row in provider_features.iterrows():
    G.add_node(provider_id_map[row['ProviderCode']], bipartite=1, num_visits=row['num_visits'], survey_score=row['survey_score'], star_rating=row['star_rating'], facility_codes=row['FacilityCode'], specialty_codes=row['SpecialtyCode'])

# Add edges with features
for _, row in edges.iterrows():
    G.add_edge(row['PERSON_ID'], row['ProviderCode'], visit_ratings=row['visit_ratings'])

# Convert to adjacency matrix
adj_matrix = nx.to_numpy_array(G)
adj_matrix = torch.tensor(adj_matrix, dtype=torch.float32)

# Extract node features for persons and providers
person_nodes = [n for n, d in G.nodes(data=True) if d['bipartite'] == 0]
provider_nodes = [n for n, d in G.nodes(data=True) if d['bipartite'] == 1]

person_features_tensor = torch.tensor([
    [d['age'], d['gender_female'], d['gender_male']] 
    for n, d in G.nodes(data=True) if n in person_nodes
], dtype=torch.float32)

provider_features_tensor = torch.tensor([
    [d['num_visits'], d['survey_score'], d['star_rating']]
    for n, d in G.nodes(data=True) if n in provider_nodes
], dtype=torch.float32)

# Create a single feature tensor
all_features = torch.cat([person_features_tensor, provider_features_tensor], dim=0)

C:\Users\samra\AppData\Local\Temp\ipykernel_23316\2725847578.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edges['PERSON_ID'] = edges['PERSON_ID'].map(person_id_map)
C:\Users\samra\AppData\Local\Temp\ipykernel_23316\2725847578.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edges['ProviderCode'] = edges['ProviderCode'].map(provider_id_map)


In [2]:
class GNNLayer(nn.Module):
    def __init__(self, in_features, out_features, dropout=0.8, l2_reg=1e-5):
        super(GNNLayer, self).__init__()
        self.linear = nn.Linear(in_features, out_features)
        self.dropout = nn.Dropout(dropout)
        self.batch_norm = nn.BatchNorm1d(out_features)
        self.l2_reg = l2_reg

    def forward(self, adjacency_matrix, feature_matrix):
        h = torch.mm(adjacency_matrix, feature_matrix)
        h = self.linear(h)
        h = self.dropout(h)
        h = self.batch_norm(h)
        return F.relu(h)

class GNNModel(nn.Module):
    def __init__(self, num_facilities, num_specialties, in_features, hidden_features, out_features, embedding_dim=16, dropout=0.8, l2_reg=1e-5):
        super(GNNModel, self).__init__()
        self.facility_embedding = nn.Embedding(num_facilities, embedding_dim)
        self.specialty_embedding = nn.Embedding(num_specialties, embedding_dim)
        self.gnn1 = GNNLayer(in_features + 2 * embedding_dim, hidden_features, dropout, l2_reg)
        self.gnn2 = GNNLayer(hidden_features, out_features, dropout, l2_reg)

    def forward(self, adjacency_matrix, features, person_facility_codes, person_specialty_codes, provider_facility_codes, provider_specialty_codes):
        # Generate embeddings for facility and specialty codes
        person_facility_embeds = [torch.mean(self.facility_embedding(torch.tensor(codes)), dim=0) for codes in person_facility_codes]
        person_specialty_embeds = [torch.mean(self.specialty_embedding(torch.tensor(codes)), dim=0) for codes in person_specialty_codes]
        provider_facility_embeds = [torch.mean(self.facility_embedding(torch.tensor(codes)), dim=0) for codes in provider_facility_codes]
        provider_specialty_embeds = [torch.mean(self.specialty_embedding(torch.tensor(codes)), dim=0) for codes in provider_specialty_codes]

        # Concatenate embeddings with other features
        person_embeds = torch.cat([torch.stack(person_facility_embeds), torch.stack(person_specialty_embeds)], dim=1)
        provider_embeds = torch.cat([torch.stack(provider_facility_embeds), torch.stack(provider_specialty_embeds)], dim=1)

        # Combine features with embeddings
        person_features = torch.cat([features[:len(person_embeds)], person_embeds], dim=1)
        provider_features = torch.cat([features[len(person_embeds):], provider_embeds], dim=1)

        features = torch.cat([person_features, provider_features], dim=0)

        # Apply GNN layers
        h = self.gnn1(adjacency_matrix, features)
        h = self.gnn2(adjacency_matrix, h)
        return h

In [3]:
person_facility_codes = [d['facility_codes'] for n, d in G.nodes(data=True) if n in person_nodes]
person_specialty_codes = [d['specialty_codes'] for n, d in G.nodes(data=True) if n in person_nodes]
provider_facility_codes = [d['facility_codes'] for n, d in G.nodes(data=True) if n in provider_nodes]
provider_specialty_codes = [d['specialty_codes'] for n, d in G.nodes(data=True) if n in provider_nodes]

In [4]:
num_facilities = len(interaction_df['FacilityCode'].unique().tolist())
num_specialties = len(interaction_df['SpecialtyCode'].unique().tolist())

In [5]:
in_feats = 3
hidden_feats = 32
out_feats = 1

In [6]:
gcn_model = GNNModel(num_facilities, num_specialties,in_feats, hidden_feats, out_feats)
gcn_model.load_state_dict(torch.load('models/gcn_reco_v2.pth'))
gcn_model.eval()

GNNModel(
  (facility_embedding): Embedding(53, 16)
  (specialty_embedding): Embedding(89, 16)
  (gnn1): GNNLayer(
    (linear): Linear(in_features=35, out_features=32, bias=True)
    (dropout): Dropout(p=0.8, inplace=False)
    (batch_norm): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (gnn2): GNNLayer(
    (linear): Linear(in_features=32, out_features=1, bias=True)
    (dropout): Dropout(p=0.8, inplace=False)
    (batch_norm): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)

In [7]:
def get_recommendations(model, user_id,features,selected_provider_lst, k=5):
    # Get the node ID for the user
    if user_id not in person_id_map:
        return []
        
    user_node = person_id_map[user_id]    
    # Get the logits (embeddings) for all nodes
    with torch.no_grad():
        logits = model(adj_matrix, features, person_facility_codes, person_specialty_codes, provider_facility_codes, provider_specialty_codes)
    
    # Get the embeddings for the user
    user_embedding = logits[user_node]
    
    # Calculate the score for each provider
    provider_scores = []
    for provider_node in provider_nodes:
        provider_embedding = logits[provider_node]
        score = torch.dot(user_embedding, provider_embedding).item()        
        # Filter based on zip code and specialty code if provided
        provider_id = id_provider_map[provider_node]
        provider_info = provider_features[provider_features['ProviderCode'] == provider_id].iloc[0]     
        
        provider_scores.append((provider_node, score))

    
    mapped_provider = []
    for item_i,item_j in provider_scores:
        if id_provider_map[item_i] in selected_provider_lst:
            mapped_provider.append((item_i,item_j))
            

    # Sort the providers by score in descending order
    provider_scores = sorted(mapped_provider, key=lambda x: x[1], reverse=True)

    # Get the top k providers
    top_providers = [(id_provider_map[provider_node],score) for provider_node, score in provider_scores[:k]]
    
    return top_providers

In [8]:
user_id = '0071588ba3116840b32d9f7fcf3ce2707c5bbcc09a3960fbb5c91e1e7e8eb21d'

In [9]:
zip_code = 34950
specialty_code = 'PS305'
selected_provider_df = interaction_df[(interaction_df['zip_code']==zip_code)
                                            & (interaction_df['SpecialtyCode']==specialty_code)]
selected_provider_lst = selected_provider_df['ProviderCode'].unique().tolist()

In [10]:
recommendations = get_recommendations(gcn_model, user_id,all_features,selected_provider_lst, k=5)
print(f"Top recommendations for user {user_id}: {recommendations}")

Top recommendations for user 0071588ba3116840b32d9f7fcf3ce2707c5bbcc09a3960fbb5c91e1e7e8eb21d: [('2J7P4', 0.486516535282135), ('XF6BG', 0.4748539626598358), ('2RTCQ', 0.45672401785850525)]


In [11]:
reco_lst = [item[0] for item in recommendations]
providers_df = interaction_df[interaction_df['ProviderCode'].isin(reco_lst)]

In [12]:
person_df=interaction_df[interaction_df['PERSON_ID']==user_id]

In [13]:
person_df

,PERSON_ID,ProviderCode,SurveyAnswerScore,RecommendStar,ratings,num_visits,age,survey_score,star_rating,zip_code,FacilityCode,SpecialtyCode,SpecialtyDescription,GENDER_Female,GENDER_Male
286,0071588ba3116840b32d9f7fcf3ce2707c5bbcc09a3960...,2BDMT,4.56,5.0,4.78,1,39.0,4.36,4.87,33063,FA22D5,PS1054,Surgical Oncology,1,0
287,0071588ba3116840b32d9f7fcf3ce2707c5bbcc09a3960...,2BDMT,4.56,5.0,4.78,1,39.0,4.36,4.87,33064,33038A,PS1054,Surgical Oncology,1,0
288,0071588ba3116840b32d9f7fcf3ce2707c5bbcc09a3960...,2BDMT,4.56,5.0,4.78,1,39.0,4.36,4.87,33065,5EC1CD,PS1054,Surgical Oncology,1,0
289,0071588ba3116840b32d9f7fcf3ce2707c5bbcc09a3960...,2BDMT,4.56,5.0,4.78,1,39.0,4.36,4.87,33316,72D9CA,PS1054,Surgical Oncology,1,0
290,0071588ba3116840b32d9f7fcf3ce2707c5bbcc09a3960...,2BDMT,4.56,5.0,4.78,1,39.0,4.36,4.87,33308,950C1E,PS1054,Surgical Oncology,1,0


In [14]:
providers_df

,PERSON_ID,ProviderCode,SurveyAnswerScore,RecommendStar,ratings,num_visits,age,survey_score,star_rating,zip_code,FacilityCode,SpecialtyCode,SpecialtyDescription,GENDER_Female,GENDER_Male
0,00123db9a36e9f9277d6ce32c3fdd1e49809845399741b...,XF6BG,2.50,5.0,3.75,1,39.0,3.09,3.79,34950,81DB7D,PS305,Family Medicine,1,0
1,32e8750a2e31a34c71d700df32a773adcab8db3109117b...,XF6BG,4.75,5.0,4.88,2,39.0,3.09,3.79,34950,81DB7D,PS305,Family Medicine,0,1
2,32e8750a2e31a34c71d700df32a773adcab8db3109117b...,XF6BG,4.67,5.0,4.83,2,39.0,3.09,3.79,34950,81DB7D,PS305,Family Medicine,0,1
3,4a2bfb67462d6f2d7b70752411114edc1325c8d9efb885...,XF6BG,4.40,5.0,4.70,1,39.0,3.09,3.79,34950,81DB7D,PS305,Family Medicine,1,0
4,c5a78302c3051b68f31cf426eff4f1b90b324d44ecdbb2...,XF6BG,4.10,5.0,4.55,2,39.0,3.09,3.79,34950,81DB7D,PS305,Family Medicine,0,1
5,c5a78302c3051b68f31cf426eff4f1b90b324d44ecdbb2...,XF6BG,1.67,1.0,1.33,2,39.0,3.09,3.79,34950,81DB7D,PS305,Family Medicine,0,1
6,df0ce9bc392f26eb5fc52244d195b5f6ab9ce6d14ddaf5...,XF6BG,2.27,2.0,2.14,1,39.0,3.09,3.79,34950,81DB7D,PS305,Family Medicine,1,0
7,e130dcef319df0dd3d6cd02df8c6295b4ec34f2a1b6281...,XF6BG,4.36,5.0,4.68,1,39.0,3.09,3.79,34950,81DB7D,PS305,Family Medicine,0,1
8,e1963c54be5e4faab8131ba53e9ff32f0c8c496223aea3...,XF6BG,2.50,5.0,3.75,2,39.0,3.09,3.79,34950,81DB7D,PS305,Family Medicine,1,0
9,e1963c54be5e4faab8131ba53e9ff32f0c8c496223aea3...,XF6BG,2.50,5.0,3.75,2,39.0,3.09,3.79,34950,81DB7D,PS305,Family Medicine,1,0


In [16]:
def compare_recommendations(person_df, provider_df, user_id, recommended_providers):
    user_data = person_df[person_df['PERSON_ID'] == user_id]
    if user_data.empty:
        print(f"No data found for user ID: {user_id}")
        return
    
    user_data = user_data.iloc[0]
    results = []

    for _, provider_data in provider_df.iterrows():
        comparison = {
            "ProviderCode": provider_data["ProviderCode"],
            "SurveyAnswerScore_diff": abs(user_data["SurveyAnswerScore"] - provider_data["SurveyAnswerScore"]),
            "RecommendStar_diff": abs(user_data["RecommendStar"] - provider_data["RecommendStar"]),
            "ratings_diff": abs(user_data["ratings"] - provider_data["ratings"]),
            "num_visits_diff": abs(user_data["num_visits"] - provider_data["num_visits"]),
            "age_diff": abs(user_data["age"] - provider_data["age"]),
            "survey_score_diff": abs(user_data["survey_score"] - provider_data["survey_score"]),
            "star_rating_diff": abs(user_data["star_rating"] - provider_data["star_rating"]),
            "same_zip_code": user_data["zip_code"] == provider_data["zip_code"],
            "same_specialty": user_data["SpecialtyCode"] == provider_data["SpecialtyCode"]
        }
        results.append(comparison)
    
    results_df = pd.DataFrame(results)
    return results_df

user_id = '0071588ba3116840b32d9f7fcf3ce2707c5bbcc09a3960fbb5c91e1e7e8eb21d'
comparison_df = compare_recommendations(person_df, providers_df, user_id, recommendations)



In [17]:
comparison_df

,ProviderCode,SurveyAnswerScore_diff,RecommendStar_diff,ratings_diff,num_visits_diff,age_diff,survey_score_diff,star_rating_diff,same_zip_code,same_specialty
0,XF6BG,2.06,0.0,1.03,0,0.0,1.27,1.08,False,False
1,XF6BG,0.19,0.0,0.10,1,0.0,1.27,1.08,False,False
2,XF6BG,0.11,0.0,0.05,1,0.0,1.27,1.08,False,False
3,XF6BG,0.16,0.0,0.08,0,0.0,1.27,1.08,False,False
4,XF6BG,0.46,0.0,0.23,1,0.0,1.27,1.08,False,False
5,XF6BG,2.89,4.0,3.45,1,0.0,1.27,1.08,False,False
6,XF6BG,2.29,3.0,2.64,0,0.0,1.27,1.08,False,False
7,XF6BG,0.20,0.0,0.10,0,0.0,1.27,1.08,False,False
8,XF6BG,2.06,0.0,1.03,1,0.0,1.27,1.08,False,False
9,XF6BG,2.06,0.0,1.03,1,0.0,1.27,1.08,False,False
